# Exploring the Allen Brain SDK
---

This is a guide notebook to explore and download the anatomical data from the Allen Brain Institute SDK.


### Installation:
You can install the SDK in a python (3.7) environment by using:

    pip install allensdk

In [5]:
import pandas as pd
import numpy as np

## Constructing a StructureTree

A StructureTree object is a wrapper around a structure graph - a list of dictionaries documenting brain structures and their containment relationships. To build a structure tree, you will first need to obtain a structure graph. The `ReferenceSpaceCache` takes care of that for you.

For a list of atlases and corresponding structure graph ids, see [here](http://help.brain-map.org/display/api/Atlas+Drawings+and+Ontologies).

In [2]:
from allensdk.core.reference_space_cache import ReferenceSpaceCache

reference_space_key = 'annotation/ccf_2017'
resolution = 25
rspc = ReferenceSpaceCache(resolution, reference_space_key, manifest='manifest.json')
# ID 1 is the adult mouse structure graph
tree = rspc.get_structure_tree(structure_graph_id=1) 

The fields contained in a StructureTree are:
    
* **acronym**: a shortened name for the structure
* **rgb_triplet**: each structure is assigned a consistent color for visualizations
* **graph_id**: the structure graph to which this structure belongs
* **graph_order**: each structure is assigned a consistent position in the flattened graph
* **id**: a unique integer identifier
* **name**: the full name of the structure
* **structure_id_path**: traces a path from the root node of the tree to this structure
* **structure_set_ids**: the structure belongs to these predefined groups

In [ ]:
# Download the CCF template at the specified resolution (npArray)
template, meta_templ = rspc.get_template_volume()
# Download the CCF annotation volume at the specified resolution (npArray)
annotation, meta_annot = rspc.get_annotation_volume()

In [ ]:
###

In [ ]:
from allensdk.api.queries.ontologies_api import OntologiesApi

oapi = OntologiesApi()

# get the ids of all the structure sets in the tree
structure_set_ids = structure_tree.get_structure_sets()

# query the API for information on those structure sets
pd.DataFrame(oapi.get_structure_sets(structure_set_ids))